In [ ]:
from pydicom import dcmread
from pydicom.pixel_data_handlers import pillow_handler
pillow_handler.PillowJPEGTransferSyntaxes.append('1.2.840.10008.1.2.4.70')
!pip install /kaggle/input/dicomsdl-offline-installer/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
try:
    import pylibjpeg
except:
    #!mkdir -p /root/.cache/torch/hub/checkpoints/
    !pip install /kaggle/input/rsna-2022-whl/{pydicom-2.3.0-py3-none-any.whl,pylibjpeg-1.4.0-py3-none-any.whl,python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl}
    #!pip install /kaggle/input/rsna-2022-whl/{torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl,torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl}

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR) 
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  #

In [ ]:
import dicomsdl as dicoml
import numpy as np 
import pandas as pd 
import tensorflow.keras as keras
import os
from multiprocess import Process, Manager
import gc
from tqdm import tqdm
import cv2
import glob
import time
import imageio as iio
#import gdcm
#import pydicom
import matplotlib.pyplot as plt
import pydicom
from pathlib import Path
from PIL import Image
import tensorflow.keras.layers as layers
import matplotlib.image as Img     
import PIL
from pydicom import dcmread
from joblib import Parallel, delayed
#import pylibjpeg
#fig, axs = plt.subplots(2, 2, figsize=(10,10))
#for ax, path in zip(axs.flat, Path('/kaggle/input/rsna-mammography-images-as-pngs/images_as_pngs_512/train_images_processed_512/10086').iterdir()):
#    img = Image.open(path)
#    ax.imshow(img, cmap="bone")
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMG_SIZE = (512,512)
BATCH_SIZE = 32
import tensorflow.keras.backend as K
import tensorflow_addons as tfa

# GPU setup

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
strategy =  tf.distribute.experimental.MultiWorkerMirroredStrategy(
    communication=tf.distribute.experimental.CollectiveCommunication.AUTO,
    cluster_resolver=None
)

# Metrics and Model

In [ ]:
class CFG:
    seed = 42
    batch_size = 128
    img_size = (512,512)
    models = 2

In [ ]:
def pfbeta_tf(labels, preds, beta=1):
    eps = 1e-5
    preds = tf.clip_by_value(preds, 0, 1)
    y_true_count = tf.reduce_sum(labels)
    ctp = tf.reduce_sum(preds[labels==1])
    cfp = tf.reduce_sum(preds[labels==0])
    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp + eps)
    c_recall = ctp / (y_true_count + eps)
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + eps)
        return result
    else:
        return tf.constant(0, dtype=tf.float32)
pfbeta_tf.__name__='pF1'


# finds best pf1 using thresholds
def pfbeta_thr(labels, preds):
    thrs = tf.range(0, 1, 0.05)
    best_score = tf.constant(0, dtype=tf.float32)
    for thr in thrs:
        score = pfbeta_tf(labels, tf.cast(preds>thr, tf.float32))
        best_score = tf.cond(score > best_score, lambda: score, lambda: best_score)
    return best_score

pfbeta_thr.__name__='pF1_thr'

def pfbeta(labels, preds, beta=1):
    eps = 1e-5
    preds = preds.clip(0, 1)
    y_true_count = labels.sum()
    ctp = preds[labels==1].sum()
    cfp = preds[labels==0].sum()
    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp + eps)
    c_recall = ctp / (y_true_count + eps)
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + eps)
        return result
    else:
        return 0.0

In [ ]:
def model(output_bias = None):

    with strategy.scope():
        inputs =  tf.keras.Input(shape=(512, 512, 3), name="input_layer",batch_size=CFG.batch_size) # shape of input image
        print(inputs.shape)
        age = tf.keras.Input(shape = (1,), dtype = tf.float32)
        base_model = tf.keras.applications.efficientnet.EfficientNetB0(
                include_top = False,
                #weights ='imagenet',
                weights=None,
        )
        #base_model.load_weights('/kaggle/input/effnet/efficientnetb0_notop.h5')
        x = base_model(inputs) 
       
        x = layers.GlobalAveragePooling2D(name="global_average_pooling_x")(x)
        
        combo = layers.Dense(32, activation = 'relu')(x)
      
        outputs = keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias)(combo)
        model = tf.keras.Model((inputs,age), outputs)

        LOSS = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)

        f1_score = tfa.metrics.F1Score(num_classes=1, threshold=0.50),
       
        Precision = tf.keras.metrics.Precision(name='Precision')
        Recall = tf.keras.metrics.Recall(name='Recall')
        #pf1 = pfbeta_tf
        #pf1_thr = pfbeta_thr
        #perda = custom_loss
        TP = tf.keras.metrics.TruePositives(thresholds = 0.5)
        FP = tf.keras.metrics.FalsePositives(thresholds = 0.5)
        TN = tf.keras.metrics.TrueNegatives(thresholds = 0.5)
        FN = tf.keras.metrics.FalseNegatives(thresholds = 0.5)
        METRICS = [Precision, Recall,f1_score,'accuracy',TP,FP,TN,FN]

        #print(METRICS)
        model.compile(loss = LOSS,
                          optimizer=tf.keras.optimizers.Adam(0.001), # use Adam with default settings
                          metrics = METRICS)
        return model

In [ ]:
%%time

final_sub_dir = Path('/kaggle/working/Submission_images')
model_1_path = '/kaggle/input/modelo-9/Model_9_simple_NoAge1.h5'
model_2_path = '/kaggle/input/modelo-9-baseline/Model_8_simple_one1 (2).h5'
model_3_path = '/kaggle/input/com-shape/Model_9_simple_NoAge0.h5'

model_1 = model()
model_1.load_weights(model_1_path)
#model_2 = model()
#model_1 = tf.keras.models.load_model(model_1_path, compile = False)
#model_2 = tf.keras.models.load_model(model_2_path , compile = False)
model_3 = model()
#model_3 = tf.keras.models.load_model(model_3_path, compile = False)
model_3.load_weights(model_3_path)

In [ ]:
model_1.layers,model_3.layers

# Create Dir with curated images

In [ ]:
test_csv = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')

In [ ]:
#test_csv['cancer'] = np.zeros(len(test_csv))
test_csv

In [ ]:
sample_csv = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/sample_submission.csv')
sample_csv

In [ ]:
!mkdir -p Submission_images

In [ ]:
test_images = glob.glob("/kaggle/input/rsna-breast-cancer-detection/test_images/*/*.dcm")

In [ ]:
%%time
#from pydicom.pixel_data_handlers.util import apply_voi_lut
submission_dir = Path('/kaggle/input/rsna-mammography-images-as-pngs/images_as_pngs_512/train_images_processed_512/10130')
RESIZE_TO = (512, 512)

  
def processing_and_saving_test_images(path, resize = RESIZE_TO ):
    
    dicom = dicoml.open(path)
    img = dicom.pixelData()
      
    img = (img - img.min()) / (img.max() - img.min())

    if dicom.PhotometricInterpretation == "MONOCHROME1":
        img = 1 - img
        
    
    
    img = cv2.resize(img, resize)
    
    img = (img * 255).astype(np.uint8)
    
    image_id =  str(path).split('/')[-1][:-4]
    
    cv2.imwrite(f'Submission_images/{image_id}.png', img)
    return

_ = Parallel(n_jobs = -1)(
    delayed(processing_and_saving_test_images)(uid)
    for uid in tqdm(test_images)
)

In [ ]:
ages = []
for dire in test_images:
    image_id = int(str(dire).split('/')[-1][:-4])
    age = test_csv[test_csv['image_id'] == image_id]['age'].ravel()[0]
    ages.append(age)
    print(image_id)
ages

In [ ]:
test_csv.head()

# Inference

In [ ]:
import tensorflow_io as tfio
import tensorflow as tf
def load_and_prep_image(filename, scale=False, log_scale = False,
                        standard_scale = False):
    # Read in the image

    img = PIL.Image.open(filename)
    img = img.convert("RGB")
    img = np.asarray(img).astype('float32')
    img = tf.convert_to_tensor(img, dtype=tf.float32)

    if scale:
        # Rescale the image (get all values between 0 and 1)
        return tf.expand_dims(img/255., axis=0)
    elif log_scale:
        img = tf.math.log(img)
        img = tf.clip_by_value(img,0,10)
        return tf.expand_dims(img, axis=0)
    elif standard_scale:
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.per_image_standardization(img)
        return img
    else:
        #print(img.shape)
        return tf.expand_dims(img, axis=0)

In [ ]:
import os
tf.get_logger().setLevel('INFO')

In [ ]:
final_dir = glob.glob('/kaggle/working/Submission_images/*')

In [ ]:
def pred_image(direct, model):
    Ids = []
    predicts = []
    contador = 0
    for dire in direct.iterdir():
        img = load_and_prep_image(dire,standard_scale = False)
        image_id = int(str(dire).split('/')[-1][:-4])
        age = test_csv[test_csv['image_id'] == image_id]['age'].ravel()[0]
        contador += 1
        #print(img.shape)
        #aa = tf.expand_dims(img, axis = 0)
        age = tf.constant([[age]])
        ds = tf.data.Dataset.from_tensor_slices((img,age))
        ds = ds.batch(1)
        ds = ds.prefetch(AUTO)
        x = next(iter(ds))
        pred_1 = model(x)
        pred = pred_1[0][0]
        ID = int(str(dire).split('/')[-1][:-4])
        Ids.append(ID)
        predicts.append(pred)
        
    pred_df = pd.DataFrame({'image_id':Ids,'cancer':predicts})
    return pred_df

In [ ]:
model_list = [model_1]#,model_2]#,model_2]

In [ ]:
%%time
df_1 = Parallel(n_jobs = -1, backend = 'threading')(
    delayed(pred_image)(final_sub_dir, model)
    for model in tqdm(model_list)
)

In [ ]:
#df_1

In [ ]:
#df_2

In [ ]:
#df_3

In [ ]:
mean = (df_1[0].cancer)/1 

In [ ]:
#mean

In [ ]:
_ = gc.collect()

# new possible sub

In [ ]:
pred_df = pd.DataFrame({'image_id':df_1[0].image_id,'cancer':mean})
pred_df.head()

In [ ]:
pred_df.cancer = pred_df.cancer.astype('float16')

In [ ]:
#test_csv.head()

In [ ]:
deli = test_csv.merge(pred_df,how = 'inner',on = 'image_id')
deli.head()

In [ ]:
deliverable = deli.groupby('prediction_id')['cancer'].mean()
deliverable = pd.DataFrame(deliverable)
deliverable['prediction_id'] = deliverable.index
deliverable.head()

In [ ]:
THRESHOLD = 0.6
deliverable["cancer"] = (deliverable["cancer"] > THRESHOLD).astype(int)
deliverable.head()

In [ ]:
entrega_2 = pd.DataFrame(data = {'prediction_id' : deliverable['prediction_id'],
                                 'cancer' : deliverable['cancer']
                                })
entrega_2 = entrega_2.reset_index(drop = True)
entrega_2.head()

In [ ]:
import shutil
shutil.rmtree("/kaggle/working/Submission_images")

In [ ]:
entrega_2.to_csv('/kaggle/working/submission.csv', index=False)